In [1]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import *
import pycuda.gpuarray as gpuarray

In [2]:
NUMEVENTS = 500
AVENUMJETS = 100

numjets = np.random.poisson(AVENUMJETS,NUMEVENTS)
stops = np.cumsum(numjets, dtype=np.int)
starts = np.zeros_like(stops)
starts[1:] = stops[:-1]
offsets = np.zeros(len(numjets)+1, dtype=np.int)
offsets[1:] = stops
data = np.random.rand(stops[-1])

In [3]:
mod = SourceModule('''

// Part that needs to be changed. Add non-zero predicate function
__device__ int predicate(float x)
{
    return (x !=0)?1:0;
}

// Counts per block
__global__ void blockCounts(float *data, int* blockcounts,int* length)
{
    unsigned int tid = threadIdx.x + blockIdx.x*blockDim.x;
    
    if (tid >= length[0])
        return;
    
    int validity = predicate(data[tid]);
    int blockcount = __syncthreads_count(validity);
    if ( threadIdx.x == 0)
        blockcounts[blockIdx.x] = blockcount;
}

// CompactK procedure. 
__global__ void compact(float* data,int* output,int* blockoffsets,int* length)
{
    unsigned int tid = threadIdx.x + blockIdx.x*blockDim.x;
    __shared__ int warpres[32];
    int warpSize = 32;
    if ( tid < length[0])
    {
        int pred = predicate(data[tid]);
		int w_i = threadIdx.x/warpSize; 
		int w_l = tid % warpSize;
		int t_m = INT_MAX >> (warpSize-w_l-1); 

		int b	= __ballot(pred) & t_m; 
		int t_u	= __popc(b);
        
        if(w_l==warpSize-1){
			warpres[w_i]=t_u+pred;
		}
        
        __syncthreads();


		if(w_i==0 && w_l<blockDim.x/warpSize){
			int w_i_u=0;
			for(int j=0;j<=5;j++){
				int b_j =__ballot( warpres[w_l] & (1<<j) );
				w_i_u += (__popc(b_j & t_m)  ) << j;
				
			}
			warpres[w_l]=w_i_u;
		}

		__syncthreads();


		if(pred){
			output[t_u+warpres[w_i]+blockoffsets[blockIdx.x]]= tid;

		}

    }
}
''')

In [4]:
func_blockcounts = mod.get_function('blockCounts')
func_compact = mod.get_function('compact')

In [5]:
from pycuda.scan import *

In [6]:
scan_kern = ExclusiveScanKernel(np.int, 'a+b', neutral=0)

In [7]:
gpu_data = gpuarray.to_gpu(data)
gpu_len = gpuarray.to_gpu(np.array([stops[-1]]).astype(np.int))
numthreads = 512
numblocks = int(np.ceil(stops[-1]/numthreads))
gpu_blockcounts = gpuarray.empty(numblocks, dtype=np.int)
gpu_output = gpuarray.empty(stops[-1], dtype=np.int)
gpu_blockoffsets = gpuarray.zeros(numblocks, dtype=np.int)

In [8]:
func_blockcounts(gpu_data,gpu_blockcounts,gpu_len, block=(numthreads,1,1), grid = (numblocks,1))

In [9]:
scan_kern(gpu_blockcounts,gpu_blockoffsets)

array([    0,   512,  1024,  1536,  2048,  2560,  3072,  3584,  4096,
        4608,  5120,  5632,  6144,  6656,  7168,  7680,  8192,  8704,
        9216,  9728, 10240, 10752, 11264, 11776, 12288, 12800, 13312,
       13824, 14336, 14848, 15360, 15872, 16384, 16896, 17408, 17920,
       18432, 18944, 19456, 19968, 20480, 20992, 21504, 22016, 22528,
       23040, 23552, 24064, 24576, 25088, 25600, 26112, 26624, 27136,
       27648, 28160, 28672, 29184, 29696, 30208, 30720, 31232, 31744,
       32256, 32768, 33280, 33792, 34304, 34816, 35328, 35840, 36352,
       36864, 37376, 37888, 38400, 38912, 39424, 39936, 40448, 40960,
       41472, 41984, 42496, 43008, 43520, 44032, 44544, 45056, 45568,
       46080, 46592, 47104, 47616, 48128, 48640, 49152, 49664])

In [10]:
func_compact(gpu_data,gpu_output,gpu_blockoffsets,gpu_len, block=(numthreads,1,1), grid=(numblocks,1))

In [11]:
np_data = np.nonzero(data)
np_data[0].shape

(49824,)

In [12]:
# Check
(gpu_output.get()==np_data[0]).all()

True